In [1]:
%reload_ext autoreload
%autoreload 2

In [3]:
from pathlib import Path
import json
import pigeonXT as pixt

### Load the job data

In [64]:
with Path('../data/job_listings.jsonl').open('r') as f:
    job_data = [json.loads(job_listing) for job_listing in f.readlines()]
job_data = [dct for dct in job_data if len(dct['text']) > 100]
print(f'{len(job_data):,} jobs loaded.')

49 jobs loaded.


### Annotate the job data

In [65]:
annotations = pixt.annotate(job_data,
                            options=['Relevant', 'Irrelevant'],
                            display_fn=lambda x: print(x['text'].replace('\n', '\n\n')))

HTML(value='0 of 49 Examples annotated, Current Position: 0 ')

Output()

### Store the job data

In [24]:
with Path('../data/job_listings.jsonl').open('w') as f:
    for _, row in annotations.iterrows():
        job_listing = row.example
        if row.changed:
            job_listing['label'] = row.label
        f.write(json.dumps(job_listing))
        f.write('\n')